# SSS

In [46]:
import random
from math import ceil
from decimal import Decimal

FIELD_SIZE = 10**7


def reconstruct_secret(shares):
	"""
	Combines individual shares (points on graph)
	using Lagranges interpolation.

	`shares` is a list of points (x, y) belonging to a
	polynomial with a constant of our key.
	"""
	sums = 0
	prod_arr = []

	for j, share_j in enumerate(shares):
		xj, yj = share_j
		prod = Decimal(1)

		for i, share_i in enumerate(shares):
			xi, _ = share_i
			if i != j:
				prod *= Decimal(Decimal(xi)/(xi-xj))

		prod *= yj
		sums += Decimal(prod)

	return int(round(Decimal(sums), 0))


def polynom(x, coefficients):
	"""
	This generates a single point on the graph of given polynomial
	in `x`. The polynomial is given by the list of `coefficients`.
	"""
	point = 0
	# Loop through reversed list, so that indices from enumerate match the
	# actual coefficient indices
	for coefficient_index, coefficient_value in enumerate(coefficients[::-1]):
		point += x ** coefficient_index * coefficient_value
	return point


def coeff(t, secret):
	"""
	Randomly generate a list of coefficients for a polynomial with
	degree of `t` - 1, whose constant is `secret`.

	For example with a 3rd degree coefficient like this:
		3x^3 + 4x^2 + 18x + 554

		554 is the secret, and the polynomial degree + 1 is
		how many points are needed to recover this secret.
		(in this case it's 4 points).
	"""
	coeff = [random.randrange(0, FIELD_SIZE) for _ in range(t - 1)]
	coeff.append(secret)
	return coeff


def generate_shares(n, m, secret):
	"""
	Split given `secret` into `n` shares with minimum threshold
	of `m` shares to recover this `secret`, using SSS algorithm.
	"""
	coefficients = coeff(m, secret)
	shares = []

	for i in range(1, n+1):
		x = random.randrange(1, FIELD_SIZE)
		shares.append((x, polynom(x, coefficients)))

	return shares


# Driver code
if __name__ == '__main__':

	# (3,5) sharing scheme
	t, n = 2, 9
	secret = 1998
	print(f'Original Secret: {secret}')

	# Phase I: Generation of shares
	shares = generate_shares(n, t, secret)
	print(f'Shares: {", ".join(str(share) for share in shares)}')

	# Phase II: Secret Reconstruction
	# Picking t shares randomly for
	# reconstruction
	pool = random.sample(shares, t)
	print(f'Combining shares: {", ".join(str(share) for share in pool)}')
	print(f'Reconstructed secret: {reconstruct_secret(pool)}')



Original Secret: 1998
Shares: (6493460, 29382192221398), (7571809, 34261602828008), (9923361, 44902116957288), (590694, 2672825375658), (645712, 2921775773678), (4883373, 22096725655968), (1144110, 5176971899898), (8947013, 40484249655568), (772041, 3493400602488)
Combining shares: (772041, 3493400602488), (645712, 2921775773678)
Reconstructed secret: 1998


# VSS

In [47]:
import random
import math

# Reference https://profs.info.uaic.ro/~siftene/Feldman.pdf

# Helper function
def isprime(n):
    if n == 2:
       return True
    if n == 1 or n % 2 == 0:
        return False    
    i = 3    
    while i <= math.sqrt(n):
        if n % i == 0:
            return False
        i = i + 2        
    return True 

# Pick q, p primes such that q | p - 1, that is equvalent to
# say that p = r*q + 1 for some r

# Choose q
while True:
    q = 127 #int(input("Insert a prime q: "))
    if isprime(q):
        break
 
print("q = " + str(q))
print("\nq is prime\n")

# Find p and r
r = 1
while True:
    p = r*q + 1
    if isprime(p):
        print("r = " + str(r))
        print("p = " + str(p))
        print("\np is prime\n")
        break
    r = r + 1

# Compute elements of Z_p*
Z_p_star = []
for i in range(0, p):
    if(math.gcd(i,p) == 1):
        Z_p_star.append(i)

print("Z_p* = ")
print(Z_p_star)

# Compute elements of G = {h^r mod p | h in Z_p*}
G = [] 
for i in Z_p_star:
    G.append(i**r % p)

G = list(set(G))
G.sort()
print("\nG = ")
print(G)
print("Order of G is " + str(len(G)) + ". This must be equal to q.")        

# Since the order of G is prime, any element of G except 1 is a generator
g = random.choice(list(filter(lambda g: g != 1, G)))
print("\ng = " + str(g))

# Secret taken from the group Z_q* 
while True:
    a0 = 84 #int(input("Inser a secret in Z_q*: "))
    if a0 >= 1 or a0 <= q:
        break

# Secret polynomium coefficients taken from the group Z_q*
a1 = random.randint(1, q)
a2 = random.randint(1, q)

a = [a0, a1, a2]

print("The secret polynomium is: " + str(a0) + " + " +  str(a1) + "x + " + str(a2) + "x^2")

# The function f is a polynomium from the group Z_q* (for simplicity 2nd degree is considered)
def f(x):
    return ((a0 + a1*x + a2*x**2) % q)

# List of shares
s = []

# Compute shares and verify
# Note that P_i receives as a share s = f(i)
# In general, each party P_1,...,P_6 could receive a different arbitrary share
for i in range(1,7):
    print("\ni = " + str(i))
    s.append(f(i))
    print("Share: f(" + str(i) + ") = " + str(f(i)))
    print("Commitment: g^f(" + str(i) + ") = " + str(g**f(i) % p))
    print("Verification: (g^a0)*((g^a1)^i)*((g^a2)^(i^2)) = " + str((g**a0)*((g**a1)**i)*((g**a2)**(i**2)) % p) + "\n") 

# Print shares
print(s)

# Parties cooperating to reconstruct the secret
B = [1,2,3]

def delta(i):
    d = 1
    print("\ni= ")
    print(i)
    for j in B:
        if j != i:
            print("\nj= ")
            print(j)
            d *= -j/(i - j)
    print("delta = " + str(d % q))
    return d

a0_reconstructed = 0
for i in B:
    print("Share of P_" + str(i) + " is " + str(s[i - 1]))
    a0_reconstructed += delta(i)*s[i - 1]

print(a0_reconstructed % q)

q = 127

q is prime

r = 4
p = 509

p is prime

Z_p* = 
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 2

In [30]:
from pvss import Pvss
from pvss.ristretto_255 import create_ristretto_255_parameters

# init, genparams
pvss_init = Pvss()
params = create_ristretto_255_parameters(pvss_init)
print(params)

# alice, genuser
pvss_alice = Pvss()
pvss_alice.set_params(params)
alice_priv, alice_pub = pvss_alice.create_user_keypair("Alice")

# boris, genuser
pvss_boris = Pvss()
pvss_boris.set_params(params)
boris_priv, boris_pub = pvss_boris.create_user_keypair("Boris")

# chris, genuser
pvss_chris = Pvss()
pvss_chris.set_params(params)
chris_priv, chris_pub = pvss_chris.create_user_keypair("Chris")

# dealer, splitsecret
pvss_dealer = Pvss()
pvss_dealer.set_params(params)
pvss_dealer.add_user_public_key(chris_pub)
pvss_dealer.add_user_public_key(alice_pub)
pvss_dealer.add_user_public_key(boris_pub)
secret0, shares = pvss_dealer.share_secret(2)

# receiver, genreceiver
pvss_receiver = Pvss()
pvss_receiver.set_params(params)
recv_priv, recv_pub = pvss_receiver.create_receiver_keypair("receiver")

# boris, reencrypt
pvss_boris.add_user_public_key(alice_pub)
pvss_boris.add_user_public_key(chris_pub)
pvss_boris.set_shares(shares)
pvss_boris.set_receiver_public_key(recv_pub)
reenc_boris = pvss_boris.reencrypt_share(boris_priv)

# alice, reencrypt
pvss_alice.add_user_public_key(boris_pub)
pvss_alice.add_user_public_key(chris_pub)
pvss_alice.set_shares(shares)
pvss_alice.set_receiver_public_key(recv_pub)
reenc_alice = pvss_alice.reencrypt_share(alice_priv)

# receiver, reconstruct
pvss_receiver.add_user_public_key(boris_pub)
pvss_receiver.add_user_public_key(chris_pub)
pvss_receiver.add_user_public_key(alice_pub)
pvss_receiver.set_shares(shares)
pvss_receiver.add_reencrypted_share(reenc_alice)
pvss_receiver.add_reencrypted_share(reenc_boris)
secret1 = pvss_receiver.reconstruct_secret(recv_priv)

print(secret0 == secret1)

b'0\x10\x06\x0c+\x06\x01\x04\x01\x83\xae\x00\x01\x00\x01\x01\x05\x00'
True


In [26]:
# init, genparams
pvss_init = Pvss()
params = create_ristretto_255_parameters(pvss_init)
users = list(range(10))
for user in users:
    

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
